<a href="https://colab.research.google.com/github/Haneenabuallam12/Haneenabuallam12/blob/main/_w5_c1_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=2ef4d3065dd99d41787d5af56db773c282cd612fd0ba079710061c10005bb376
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
!Pip install sklearn

/bin/bash: Pip: command not found


In [4]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np 
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [5]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".
start = dt.datetime(2020,9,22)
end = dt.datetime.now()

stock_symbol = ['AVAX-USD']

In [6]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

deletd_column =[]
df = pdr.get_data_yahoo(stock_symbol, start, end).dropna()
df
scaler = StandardScaler()

scaled_df = pd.DataFrame(
    scaler.fit_transform(df), 
    columns=df.columns,
    index=df.index)

scaled_df= scaled_df.drop("Close",axis="columns")

scaled_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Adj Close,Volume
Date,,,,,
2020-09-22,-0.948016,-0.682762,-0.966446,-0.936332,-0.181809
2020-09-23,-0.936274,-0.954487,-1.000378,-1.016810,-0.483817
2020-09-24,-1.017556,-0.976769,-0.999961,-0.967167,-0.534523
2020-09-25,-0.967087,-0.970847,-0.960935,-0.960913,-0.609622
2020-09-26,-0.960422,-0.977903,-0.953540,-0.968914,-0.663659
...,...,...,...,...,...
2021-11-06,2.342108,2.301463,2.453179,2.514774,0.186414
2021-11-07,2.556161,2.752222,2.734153,2.938593,1.728221
2021-11-08,2.978646,3.110671,3.013578,3.102073,2.898804


In [7]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

scaled_df['Close_15_Rolling'] = scaled_df['Adj Close'].rolling(16).mean() #start from zero ?

scaled_df['Open_1_Change'] = scaled_df['Open'].diff()

scaled_df['Adj Close_Change'] = scaled_df['Adj Close'].diff(-1)

scaled_df['dif_high_low']= scaled_df['High']-scaled_df['Low']

scaled_df['Volume_Change'] = scaled_df['Volume'].diff()



scaled_df=scaled_df.dropna()
X=scaled_df
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,Adj Close_Change,dif_high_low,Volume_Change
Date,,,,,,,,,,
2020-10-11,-0.993792,-1.007056,-0.974173,-0.989837,-0.693197,-0.985261,-0.010421,-0.002546,-0.032883,-0.005702
2020-10-14,-0.971886,-0.992541,-0.971588,-0.987291,-0.695878,-0.988446,0.021905,0.001067,-0.020953,-0.002681
2020-10-15,-0.987597,-1.004612,-0.971901,-0.988358,-0.701433,-0.986668,-0.015711,0.004082,-0.032711,-0.005556
2020-10-16,-0.988787,-1.007193,-0.978365,-0.992440,-0.696273,-0.988247,-0.001190,-0.007308,-0.028829,0.005160
2020-10-17,-0.993027,-1.005390,-0.973880,-0.985132,-0.703070,-0.989761,-0.004240,0.003756,-0.031510,-0.006797
...,...,...,...,...,...,...,...,...,...,...
2021-11-05,2.567634,2.407526,2.450127,2.311776,0.382505,1.973393,0.164718,-0.202999,-0.042600,-1.009675
2021-11-06,2.342108,2.301463,2.453179,2.514774,0.186414,2.024418,-0.225526,-0.423819,-0.151716,-0.196092
2021-11-07,2.556161,2.752222,2.734153,2.938593,1.728221,2.089757,0.214053,-0.163479,0.018069,1.541807


In [8]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = X["Adj Close_Change"]
y = np.where(y<0,-1,1)
y.shape
X.shape
X=X.drop("Adj Close_Change",axis="columns")
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,dif_high_low,Volume_Change
Date,,,,,,,,,
2020-10-11,-0.993792,-1.007056,-0.974173,-0.989837,-0.693197,-0.985261,-0.010421,-0.032883,-0.005702
2020-10-14,-0.971886,-0.992541,-0.971588,-0.987291,-0.695878,-0.988446,0.021905,-0.020953,-0.002681
2020-10-15,-0.987597,-1.004612,-0.971901,-0.988358,-0.701433,-0.986668,-0.015711,-0.032711,-0.005556
2020-10-16,-0.988787,-1.007193,-0.978365,-0.992440,-0.696273,-0.988247,-0.001190,-0.028829,0.005160
2020-10-17,-0.993027,-1.005390,-0.973880,-0.985132,-0.703070,-0.989761,-0.004240,-0.031510,-0.006797
...,...,...,...,...,...,...,...,...,...
2021-11-05,2.567634,2.407526,2.450127,2.311776,0.382505,1.973393,0.164718,-0.042600,-1.009675
2021-11-06,2.342108,2.301463,2.453179,2.514774,0.186414,2.024418,-0.225526,-0.151716,-0.196092
2021-11-07,2.556161,2.752222,2.734153,2.938593,1.728221,2.089757,0.214053,0.018069,1.541807


In [9]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=0)

y_test.shape

(118,)

In [10]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [11]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.4033300725248158]
1,High,[0.046566541905853215]
2,Low,[-0.11412994021469869]
3,Adj Close,[-0.10244717086410991]
4,Volume,[-0.16245759680083666]
5,Close_15_Rolling,[-0.12389185458327437]
6,Open_1_Change,[-0.14413719635631514]
7,dif_high_low,[0.1606964821205529]
8,Volume_Change,[0.07059508287541184]


In [12]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
predictions

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1])

In [13]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.
cnf_matrix = metrics.confusion_matrix(y_test, predictions)
cnf_matrix
#TODO

array([[57,  7],
       [50,  4]])

In [14]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test,predictions)
accuracy

0.5169491525423728

In [15]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".
cross_val = cross_val_score(logistic,X, y, cv=5)
cross_val

array([0.53164557, 0.48101266, 0.44303797, 0.42307692, 0.3974359 ])